In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import datetime
import numpy as np

In [34]:
# Read the dataset from CSV
df = pd.read_csv('Food.csv')

In [35]:
df

,User ID,Food Name,Cuisine Type\t,Dietary Restrictions,Flavor Profiles,Allergies,Time of Day,Occasion,Budget
0,1,Italian Cuisine,Italian,Vegetarian,Savory,NaN,Dinner,Casual,Moderate
1,2,Asian Cuisine,Japanese,NaN,Umami,NaN,Lunch,Casual,Moderate
2,3,Italian Cuisine,Italian,Gluten-Free,Savory,NaN,Dinner,Formal,High
3,4,Indian Cuisine,Indian,NaN,Spicy,NaN,Lunch,Casual,Moderate
4,5,Indian Cuisine,Indian,Vegan,Fresh,NaN,Lunch,Casual,Low
...,...,...,...,...,...,...,...,...,...
206,208,Indian Cuisine,Indian,NaN,"Sweet, Crunchy",NaN,Snack,Casual,Moderate
207,209,Indian Cuisine,Indian,Vegetarian,Creamy,Dairy,Dinner,Casual,Moderate
208,210,Indian Cuisine,Indian,NaN,Smoky,NaN,Dinner,Casual,Moderate
209,211,Indian Cuisine,Indian,Vegetarian,Creamy,Nuts,Dinner,Special,Moderate


In [36]:
# Splitting the data into train and test sets
X = df[['Cuisine Type\t', 'Dietary Restrictions', 'Flavor Profiles', 'Allergies', 'Budget',
        'Time of Day', 'Occasion']]
y = df[['Food Name']]


In [37]:
# Label encoding for the target variable
label_encoder = LabelEncoder()
y['Food Name'] = label_encoder.fit_transform(y['Food Name'])

C:\Users\Kalana\AppData\Local\Temp\ipykernel_18152\3270170203.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Food Name'] = label_encoder.fit_transform(y['Food Name'])


In [38]:
# Applying one-hot encoding to categorical features and target variable
cat_features = ['Cuisine Type\t', 'Dietary Restrictions', 'Flavor Profiles', 'Allergies', 'Budget',
        'Time of Day', 'Occasion']
preprocessor = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), cat_features)],
    remainder='passthrough')

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

In [41]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [42]:
# Training the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

c:\Users\Kalana\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=42)

In [43]:
# Make predictions on the test set
y_pred = model.predict(X_test_encoded)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Get the current date and time
current_datetime = datetime.datetime.now()
print("Current Date and Time:", current_datetime)

Mean Squared Error: 0.0
Current Date and Time: 2023-09-04 03:54:00.641247


In [44]:
y_pred

array([0., 0., 0., 4., 0., 1., 0., 1., 0., 4., 0., 1., 0., 0., 2., 1., 0.,
       2., 1., 4., 4., 1., 3., 1., 3., 0., 1., 4., 0., 1., 0., 1., 0., 4.,
       1., 3., 4., 4., 3., 4., 1., 1., 4.])

In [45]:
# Convert the predicted values back to their original form
predicted_y = pd.DataFrame({'Food_Name': label_encoder.inverse_transform(np.round(y_pred).astype(int))})

In [46]:
predicted_y

,Food_Name
0,Asian Cuisine
1,Asian Cuisine
2,Asian Cuisine
3,Sri Lankan Cuisine
4,Asian Cuisine
5,Indian Cuisine
6,Asian Cuisine
7,Indian Cuisine
8,Asian Cuisine
9,Sri Lankan Cuisine


In [47]:
import pickle

with open('fine_dining.dat', 'wb') as f:
    pickle.dump(model, f)

In [48]:
with open('fine_dining.dat' , 'rb') as f:
    fine_dining_model = pickle.load(f)

In [53]:
import pandas as pd
data=preprocessor.transform(pd.DataFrame([['Italian','Vegetarian','Savory','NaN','Dinner','Casual','Moderate']], columns=['Cuisine Type\t', 'Dietary Restrictions', 'Flavor Profiles', 'Allergies', 'Budget',
        'Time of Day', 'Occasion']))
new_data=fine_dining_model.predict(data)
label_encoder.inverse_transform(np.round(new_data).astype(int))

array(['Italian Cuisine'], dtype=object)